# Review Instructions

Please review the MSv4 `antenna_xds` schema and the XRADIO interface (`ps['MSv4_name'].antenna_xds`). Note that the PS (processing set) interface or the main_xds should not be reviewed.

The `antenna_xds` schema specification: https://docs.google.com/spreadsheets/d/14a6qMap9M5r_vjpLnaBKxsR9TF4azN5LVdOxLacOX-s/edit#gid=257301047

The processing set is a loose collection of MSv4 which might come from multiple MSv2 (or ASDMS). Consequently, arbitrary ids are avoided in favor of descriptive strings.

Two example datasets will be used VLBA_TL016B_split_lsrk.ms (VLBI) and uid___A002_X1015532_X1926f.small.ms (Single Dish).

## Preparatory Material
Go over Xarray nomenclature and selection syntax:
- https://docs.xarray.dev/en/latest/user-guide/terminology.html
- https://docs.xarray.dev/en/latest/user-guide/indexing.html

MSv2 and CASA documentation:
- MSv2 schema: https://casacore.github.io/casacore-notes/229.pdf
- MSv3 schema: https://casacore.github.io/casacore-notes/264.pdf

VLBI MSv2 extension: 
https://casacore.github.io/casacore-notes/265.pdf

## `antenna_xds` Schema
The ANTENNA, FEED, GAIN_CURVE, PHASE_CAL, and INTERFEROMETER_MODEL (VLBI, not yet included) tables in the MSv2 contain closely related information that is all related to the antenna. The MSv4 contains a single SPW, consequently, an antenna will only have a single feed associated with it.

Antenna_ids are no longer used in favor of the antenna name (for antennas that can move the name consists of the name + "_" + station). This allows for easily comparing MSv4s from different observations.

## Key Questions to Answer
### Schema Questions
- 1.1) Are there missing use cases?
- 1.2) Is all the information present needed for offline processing?
- 1.3) (VLBI) Instead of storing BASELINE_REFERENCE in main_xds can we store it in the antenna_xds? This would assume that for the duration of the MS v4 that the reference antennas remain constant.
- 1.4) Is the order of the dims correct (antenna_id)?
- 1.5) Should BEAM_OFFSET be sky_dir_label (Ra, Dec) or local_sky_label (Az, Alt)?
- 1.6) Should we add prefex to organize data variables? For example PHASE_DELAY -> VLBI_PHASE_DELAY?
- 1.7) Should we include the POLARIZATION_RESPONSE doesn't seem to be used?
- 1.8) Can we get rid of POLARIZATION_RESPONSE?
- 1.9) ANTENNA_FEED_OFFSET is a new data variable that is calculated using (Antenna_Table.offset + Feed_Table.position). Is this any need to store these values separately?

### XRADIO
2.1) Please check all the data variable (names,dimensions,measures) and coordinates (names,dimensions,measures) in both the google spreadsheet and ipynb.

# Environment instructions

It is recommended to use the conda environment manager to create a clean, self-contained runtime where xradio and all its dependencies can be installed:

```bash
conda create --name xradio python=3.11 --no-default-packages
conda activate xradio
```

Clone the repository, checkout the review branch and do a local install:

```bash
git clone https://github.com/casangi/xradio.git
git checkout 168-review-ms_xdsattrsantenna_xds-schema-and-xradio-interface
cd xradio
pip install -e .
```

On macOS it is required to pre-install python-casacore using ```bash conda install -c conda-forge python-casacore```.

# Data
Two examples will be given VLBA_TL016B_split_lsrk.ms (VLBI) and  uid___A002_X1015532_X1926f.small.ms (Single Dish). But you are welcome to change things up by downloading (or using your own MSv2 dataset). The GraphVIPER function list_files will give you a list of available datasets.


In [11]:
from graphviper.utils.data import list_files
list_files()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ file                                     ┃ dtype          ┃ telescope   ┃ size     ┃ mode                       ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ ea25_cal_small_before_fixed.split.ms     │ CASA MS v2     │ VLA         │ 0.209    │ Holography                 │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ ea25_cal_small_after_fixed.split.ms      │ CASA MS v2     │ VLA         │ 0.209    │ Holography                 │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ J1924-2914.ms.calibrated.split.SPW3      │ CASA MS v2     │ ALMA        │ 0.055    │ Holography                 │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ extract_holog_verification.json          │ json           │ na          │ 0.000004 │ na                         │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ holog_numerical_verification.json        │ json           │ na          │ 0.000004 │ na                         │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ locit-input-pha.cal                      │ CASA Cal Table │ VLA         │ 0.000468 │ antenna location cal table │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ panel_cutoff_mask                        │ npy            │ na          │ 0.000348 │ na                         │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ heuristic_model                          │ bin            │ na          │ 0.000004 │ na                         │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ panel_test_files                         │ na             │ na          │          │ na                         │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ AA2-Mid-sim_00000.ms                     │ CASA MS v2     │ SKA Mid     │ 0.029    │ Simulated Interferometer   │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ Antennae_M8.img.zarr                     │ zarr           │ ALMA        │ 0.014    │ Interferometer Mosaic      │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ Antennae_North.cal.lsrk.ms               │ CASA MS v2     │ ALMA        │ 0.202    │ Interferometer Mosaic      │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ Antennae_North.cal.lsrk.vis.zarr         │ zarr           │ ALMA        │ 0.207    │ Interferometer Mosaic      │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ Antennae_North.cal.lsrk.split.ms         │ CASA MS v2     │ ALMA        │ 0.0056   │ Interferometer Mosaic      │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ Antennae_North.cal.lsrk.split.vis.zarr   │ zarr           │ ALMA        │ 0.0047   │ Interferometer Mosaic      │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ complex_valued_uv.im                     │ image          │ Simulated   │ 0.768    │ Simulated                  │
├──────────────────────────────────────────┼────────────

# VLBI Example

## Download Dataset

In [1]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set
from xradio.vis.read_processing_set import read_processing_set
import graphviper

graphviper.utils.data.download(file="VLBA_TL016B_split_lsrk.ms")

[2024-08-08 15:34:55,851]  WARNING  graphviper:  File exists: /Users/jsteeb/Downloads/yes/envs/zinc/lib/python3.11/site-packages/graphviper/utils/data/.dropbox 
[2024-08-08 15:34:55,852]     INFO  graphviper:  Updating file metadata information ...  
 

 Download List              
 ─────────────────────────── 
  VLBA_TL016B_split_lsrk.ms

[2024-08-08 15:34:56,770]     INFO  graphviper:  File exists: VLBA_TL016B_split_lsrk.ms 


## Start Dask cluster 
Choose an approriate number of cores and memory_limit (this is per core).

In [2]:
from graphviper.dask.client import local_client

viper_client = local_client(cores=4, memory_limit="4GB")
viper_client

[2024-08-08 15:34:56,853]     INFO  graphviper:  Checking parameter values for client.local_client 
[2024-08-08 15:34:56,854]     INFO  graphviper:  Module path: /Users/jsteeb/Downloads/yes/envs/zinc/lib/python3.11//site-packages/ 
[2024-08-08 15:34:56,857]  WARNING      client:  It is recommended that the local cache directory be set using the dask_local_dir parameter. 
[2024-08-08 15:34:57,573]     INFO      client:  Created client <MenrvaClient: 'tcp://127.0.0.1:51975' processes=4 threads=4, memory=14.90 GiB> 


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:51975,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 14.90 GiB
Comm: tcp://127.0.0.1:51987,Total threads: 1
Dashboard: http://127.0.0.1:51989/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:51978,


## Convert dataset

In [3]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

in_file = "VLBA_TL016B_split_lsrk.ms"
out_file = "VLBA_TL016B_split_lsrk.vis.zarr"

convert_msv2_to_processing_set(
    in_file=in_file,
    out_file=out_file,
    parallel=True,
    overwrite=True,
)

[2024-08-08 15:34:57,609]     INFO      client:  Partition scheme that will be used: ['DATA_DESC_ID', 'OBSERVATION_ID', 'FIELD_ID'] 
[2024-08-08 15:34:57,615]     INFO      client:  Number of partitions: 4 
[2024-08-08 15:34:57,615]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [-1], FIELD [0], SCAN [0] 
[2024-08-08 15:34:57,785]  WARNING      client:  Source_id is -1. No source information will be included in the field_and_source_xds. 
[2024-08-08 15:34:57,859]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [-1], FIELD [1], SCAN [0] 
[2024-08-08 15:34:58,067]  WARNING      client:  Source_id is -1. No source information will be included in the field_and_source_xds. 
[2024-08-08 15:34:58,161]     INFO      client:  OBSERVATION_ID [0], DDI [1], STATE [-1], FIELD [0], SCAN [0] 
[2024-08-08 15:34:58,350]  WARNING      client:  Source_id is -1. No source information will be included in the field_and_source_xds. 
[2024-08-08 15:34:58,429]     INFO      client:  OBSERV

## Inspect Processing Set

In [5]:
import pandas as pd

# Set the maximum number of rows displayed before scrolling
pd.set_option("display.max_rows", 1000)

from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set("VLBA_TL016B_split_lsrk.vis.zarr")
ps.summary()

,name,obs_mode,shape,polarization,spw_name,field_name,source_name,field_coords,start_frequency,end_frequency
0,VLBA_TL016B_split_lsrk_3,obs_0,"(540, 55, 6, 2)","[RR, LL]",spw_1,[J1154+6022_1],[Unknown],"[fk5, 11h54m04.54s, 60d22m20.82s]",5.068199e+09,5.070699e+09
1,VLBA_TL016B_split_lsrk_2,obs_0,"(200, 55, 6, 2)","[RR, LL]",spw_1,[4C39.25_0],[Unknown],"[fk5, 9h27m03.01s, 39d02m20.85s]",5.068199e+09,5.070699e+09
2,VLBA_TL016B_split_lsrk_0,obs_0,"(200, 55, 6, 2)","[RR, LL]",spw_0,[4C39.25_0],[Unknown],"[fk5, 9h27m03.01s, 39d02m20.85s]",5.004196e+09,5.006697e+09
3,VLBA_TL016B_split_lsrk_1,obs_0,"(540, 55, 6, 2)","[RR, LL]",spw_0,[J1154+6022_1],[Unknown],"[fk5, 11h54m04.54s, 60d22m20.82s]",5.004196e+09,5.006697e+09


## Inspect antenna_xds:

In [6]:
ant_xds = ps['VLBA_TL016B_split_lsrk_0'].attrs['antenna_xds'].load()
ant_xds

<xarray.Dataset> Size: 521kB
Dimensions:                   (name: 10, cartesian_pos_label: 3,
                               receptor_name: 2, sky_dir_label: 2,
                               gain_curve_time: 1, poly_term: 1,
                               antenna_id: 10, phase_cal_time: 642,
                               tone_label: 2)
Coordinates:
  * cartesian_pos_label       (cartesian_pos_label) <U1 12B 'x' 'y' 'z'
  * gain_curve_time           (gain_curve_time) float64 8B 5.152e+09
    mount                     (name) <U6 240B 'ALT-AZ' 'ALT-AZ' ... 'ALT-AZ'
  * name                      (name) <U2 80B 'BR' 'FD' 'HN' ... 'OV' 'PT' 'SC'
  * phase_cal_time            (phase_cal_time) float64 5kB 5.152e+09 ... 5.15...
  * receptor_name             (receptor_name) <U21 168B '0' '1'
  * sky_dir_label             (sky_dir_label) <U3 24B 'ra' 'dec'
    station                   (name) <U2 80B 'BR' 'FD' 'HN' ... 'OV' 'PT' 'SC'
Dimensions without coordinates: poly_term, antenna_id, tone_label
Data variables: (12/13)
    ANTENNA_DISH_DIAMETER     (name) float64 80B 25.0 25.0 25.0 ... 25.0 25.0
    ANTENNA_FEED_OFFSET       (name, cartesian_pos_label) float64 240B 2.13 ....
    ANTENNA_POSITION          (name, cartesian_pos_label) float64 240B -2.112...
    BEAM_OFFSET               (name, receptor_name, sky_dir_label) float64 320B ...
    GAIN_CURVE                (name, gain_curve_time, poly_term, receptor_name) float32 80B ...
    GAIN_CURVE_INTERVAL       (name, gain_curve_time) float64 80B 2.601e+04 ....
    ...                        ...
    PHASE_CAL                 (antenna_id, phase_cal_time, receptor_name, tone_label) complex64 205kB ...
    PHASE_CAL_CABLE_CAL       (antenna_id, phase_cal_time) float64 51kB 1.507...
    PHASE_CAL_INTERVAL        (antenna_id, phase_cal_time) float64 51kB 30.0 ...
    PHASE_CAL_TONE_FREQUENCY  (antenna_id, phase_cal_time, receptor_name, tone_label) float64 205kB ...
    POLARIZATION_TYPE         (name, receptor_name) <U1 80B 'R' 'L' ... 'R' 'L'
    RECEPTOR_ANGLE            (name, receptor_name) float64 160B 0.0 0.0 ... 0.0
Attributes:
    overall_telescope_name:  VLBA
    relocatable_antennas:    False

# Single Dish Example

## Convert Dataset

In [15]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set
from xradio.vis.read_processing_set import read_processing_set
import graphviper

graphviper.utils.data.download(file="uid___A002_X1015532_X1926f.small.ms")

[2024-08-08 16:15:32,993]  WARNING      client:  File exists: /Users/jsteeb/Downloads/yes/envs/zinc/lib/python3.11/site-packages/graphviper/utils/data/.dropbox 
[2024-08-08 16:15:32,994]     INFO      client:  Updating file metadata information ...  
 

 Download List                        
 ───────────────────────────────────── 
  uid___A002_X1015532_X1926f.small.ms

uid___A002_X1015532_X1926f.small.ms.zip:   0%|          | 0.00/6.01M [00:00<?, ?iB/s]

## Convert Dataset

In [16]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

in_file = "uid___A002_X1015532_X1926f.small.ms"
out_file = "uid___A002_X1015532_X1926f.small.vis.zarr"

convert_msv2_to_processing_set(
    in_file=in_file,
    out_file=out_file,
    parallel=True,
    overwrite=True,
)

[2024-08-08 16:16:46,110]     INFO      client:  Partition scheme that will be used: ['DATA_DESC_ID', 'OBS_MODE', 'OBSERVATION_ID', 'FIELD_ID'] 
[2024-08-08 16:16:46,142]     INFO      client:  Number of partitions: 20 
[2024-08-08 16:16:46,142]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [10], FIELD [1], SCAN [2 4] 
[2024-08-08 16:16:46,143]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [11], FIELD [1], SCAN [2 4] 
[2024-08-08 16:16:46,143]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [12], FIELD [1], SCAN [2 4] 
[2024-08-08 16:16:46,143]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [13 15 17 19 21 23 25 27 29 31 33 35 37 39 41 43 45], FIELD [1], SCAN [3 5] 
[2024-08-08 16:16:46,144]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [14 16 18 20 22 24 26 28 30 32 34 36 38 40 42 44], FIELD [1], SCAN [3 5] 
[2024-08-08 16:16:46,144]     INFO      client:  OBSERVATION_ID [0], DDI [1], STATE [10], FIELD [1], SCAN [2 4] 
[2024-08-

## Inspect Processing Set

In [18]:
import pandas as pd

# Set the maximum number of rows displayed before scrolling
pd.set_option("display.max_rows", 1000)

from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set("uid___A002_X1015532_X1926f.small.vis.zarr")
ps.summary()

,name,obs_mode,shape,polarization,spw_name,field_name,source_name,field_coords,start_frequency,end_frequency
0,uid___A002_X1015532_X1926f.small_9,OBSERVE_TARGET#ON_SOURCE,"(864, 4, 4, 2)","[XX, YY]",X780709176#ALMA_RB_07#BB_2#SW-01#FULL_RES_1,[Jupiter_1],[Jupiter_1],Ephemeris,3.450297e+11,3.450765e+11
1,uid___A002_X1015532_X1926f.small_7,"CALIBRATE_ATMOSPHERE#HOT,CALIBRATE_WVR#HOT","(30, 4, 4, 2)","[XX, YY]",X780709176#ALMA_RB_07#BB_2#SW-01#FULL_RES_1,[Jupiter_1],[Jupiter_1],Ephemeris,3.450297e+11,3.450765e+11
2,uid___A002_X1015532_X1926f.small_0,"CALIBRATE_ATMOSPHERE#OFF_SOURCE,CALIBRATE_WVR#...","(30, 4, 4, 2)","[XX, YY]",X780709176#ALMA_RB_07#BB_1#SW-01#FULL_RES_0,[Jupiter_1],[Jupiter_1],Ephemeris,3.457649e+11,3.457650e+11
3,uid___A002_X1015532_X1926f.small_19,OBSERVE_TARGET#ON_SOURCE,"(864, 4, 4, 2)","[XX, YY]",X780709176#ALMA_RB_07#BB_4#SW-01#FULL_RES_3,[Jupiter_1],[Jupiter_1],Ephemeris,3.544732e+11,3.544736e+11
4,uid___A002_X1015532_X1926f.small_10,"CALIBRATE_ATMOSPHERE#OFF_SOURCE,CALIBRATE_WVR#...","(30, 4, 4, 2)","[XX, YY]",X780709176#ALMA_RB_07#BB_3#SW-01#FULL_RES_2,[Jupiter_1],[Jupiter_1],Ephemeris,3.551176e+11,3.551190e+11
5,uid___A002_X1015532_X1926f.small_17,"CALIBRATE_ATMOSPHERE#HOT,CALIBRATE_WVR#HOT","(30, 4, 4, 2)","[XX, YY]",X780709176#ALMA_RB_07#BB_4#SW-01#FULL_RES_3,[Jupiter_1],[Jupiter_1],Ephemeris,3.544732e+11,3.544736e+11
6,uid___A002_X1015532_X1926f.small_1,"CALIBRATE_ATMOSPHERE#AMBIENT,CALIBRATE_WVR#AMB...","(30, 4, 4, 2)","[XX, YY]",X780709176#ALMA_RB_07#BB_1#SW-01#FULL_RES_0,[Jupiter_1],[Jupiter_1],Ephemeris,3.457649e+11,3.457650e+11
7,uid___A002_X1015532_X1926f.small_6,"CALIBRATE_ATMOSPHERE#AMBIENT,CALIBRATE_WVR#AMB...","(30, 4, 4, 2)","[XX, YY]",X780709176#ALMA_RB_07#BB_2#SW-01#FULL_RES_1,[Jupiter_1],[Jupiter_1],Ephemeris,3.450297e+11,3.450765e+11
8,uid___A002_X1015532_X1926f.small_8,OBSERVE_TARGET#OFF_SOURCE,"(510, 4, 4, 2)","[XX, YY]",X780709176#ALMA_RB_07#BB_2#SW-01#FULL_RES_1,[Jupiter_1],[Jupiter_1],Ephemeris,3.450297e+11,3.450765e+11
9,uid___A002_X1015532_X1926f.small_16,"CALIBRATE_ATMOSPHERE#AMBIENT,CALIBRATE_WVR#AMB...","(30, 4, 4, 2)","[XX, YY]",X780709176#ALMA_RB_07#BB_4#SW-01#FULL_RES_3,[Jupiter_1],[Jupiter_1],Ephemeris,3.544732e+11,3.544736e+11


## Inspect antenna_xds

In [20]:
ant_xds = ps['uid___A002_X1015532_X1926f.small_9'].attrs['antenna_xds'].load()
ant_xds

<xarray.Dataset> Size: 956B
Dimensions:                (name: 4, cartesian_pos_label: 3, receptor_name: 2,
                            sky_dir_label: 2)
Coordinates:
  * cartesian_pos_label    (cartesian_pos_label) <U1 12B 'x' 'y' 'z'
    mount                  (name) <U6 96B 'ALT-AZ' 'ALT-AZ' 'ALT-AZ' 'ALT-AZ'
  * name                   (name) <U9 144B 'PM01_T703' ... 'PM04_T704'
  * receptor_name          (receptor_name) <U21 168B '0' '1'
  * sky_dir_label          (sky_dir_label) <U3 24B 'ra' 'dec'
    station                (name) <U4 64B 'T703' 'T701' 'T702' 'T704'
Data variables:
    ANTENNA_DISH_DIAMETER  (name) float64 32B 12.0 12.0 12.0 12.0
    ANTENNA_FEED_OFFSET    (name, cartesian_pos_label) float64 96B 0.0 ... 0.0
    ANTENNA_POSITION       (name, cartesian_pos_label) float64 96B 2.225e+06 ...
    BEAM_OFFSET            (name, receptor_name, sky_dir_label) float64 128B ...
    POLARIZATION_TYPE      (name, receptor_name) <U1 32B 'X' 'Y' 'X' ... 'X' 'Y'
    RECEPTOR_ANGLE         (name, receptor_name) float64 64B -0.9346 ... 0.6362
Attributes:
    overall_telescope_name:  ALMA
    relocatable_antennas:    True